In [194]:
import os, shutil
HOME = 'C:/Users/park0/yolov11/customData/'
backupPath = os.path.join(HOME, 'backup')

# 기존의 폴더가 있으면 삭제 후 다시 생성
orgDataPath =  os.path.join(HOME, 'org')
if os.path.exists(orgDataPath):
    shutil.rmtree(orgDataPath)
shutil.copytree(backupPath, orgDataPath)

'C:/Users/park0/yolov11/customData/org'

## 데이터셋 확인

In [195]:
from glob import glob 

# 원본 데이터셋이 저장된 폴더
imagefiles = glob(os.path.join(orgDataPath, '*.JPG'))
labelfiles = glob(os.path.join(orgDataPath, '*.txt'))

In [196]:
# 이미지 파일과 txt파일이 동일하게 한쌍으로 존재하는지 확인
correct = 0
for i in range(len(imagefiles)):
    if imagefiles[i].split('.JPG')==labelfiles[i].split('.txt'):
        correct +=1
        #print(imagefiles[i], labelfiles[i])
print("correct: {}/{}".format(correct,len(imagefiles)))

correct: 4/4


In [197]:
import random
import cv2
from matplotlib import pyplot as plt
import albumentations as A

In [198]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, width, height, dataType='yolo', color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""

    if dataType == 'coco':
        x_min, y_min, w, h = bbox
        
        x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)

    elif dataType =='yolo':
        x_center, y_center, w, h = bbox
        # 정규화된 값 0~1사이의 값
        # 픽셀 좌표로 변환 이미지의 width와 height값을 곱해주면 된다.
        x_min = int((x_center - (w/2))*width)
        x_max = int((x_center + (w/2))*width)
        y_min = int((y_center - (h/2))*height)
        y_max = int((y_center + (h/2))*height)
    print(x_min, x_max, y_min,y_max)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35,
        color=TEXT_COLOR,
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name, img_width, img_height):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name, img_width, img_height, 'yolo')
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [199]:
imagefiles

['C:/Users/park0/yolov11/customData/org\\chocoChip.JPG',
 'C:/Users/park0/yolov11/customData/org\\gomet.JPG',
 'C:/Users/park0/yolov11/customData/org\\potatoCookie.JPG',
 'C:/Users/park0/yolov11/customData/org\\sandAce.JPG']

In [200]:
def read_label_txt(txtFile):
    category_ids = []
    bboxes = []

    f=open(txtFile,'r')

    while True:
        line = f.readline()
        if not line: break
        ids, xc, yc, w, h= line.split(' ')
        category_ids.append(int(ids))
        bboxes.append([float(xc),float(yc),float(w),float(h)])
        #print(line)
    f.close()
    return category_ids, bboxes

In [201]:
def write_label_txt(txtFile, category_ids, bboxes):
    f=open(txtFile,'w')

    for i, ids in enumerate(category_ids):
        xc,yc,w,h = bboxes[i]
        f.write("{} {} {} {} {}\n".format(int(ids),xc,yc,w,h))
        #print("{} {}".format(int(ids), bboxes[i]))
    f.close()

In [202]:
category_id_to_name = {0: 'chocoChip', 1: 'gomet', 2: 'potatoCookie', 3: 'sandAce'}

### 원본 이미지를 축소

In [203]:
transform_resize = A.Compose(
    [A.LongestMaxSize(max_size=800, interpolation=cv2.INTER_LINEAR),],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

### 원본 이미지 좌우 대칭

In [204]:
transform_hflip = A.Compose(
    [A.HorizontalFlip(p=1.0)],
    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids']),
)

In [205]:
transform_hsv = A.HueSaturationValue(
    hue_shift_limit=(-5,5),
    sat_shift_limit=(20,20),
    val_shift_limit=(30,30),
    p=0.7
)

In [206]:
transform_shift = A.Compose(
    [A.ShiftScaleRotate(shift_limit=(-0.2, 0.2), scale_limit=(0, 0), rotate_limit=(0, 0),p=1.0)],
    bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.2, label_fields=['category_ids']),
)

In [207]:
transform_scale = A.Compose(
    [A.ShiftScaleRotate(shift_limit=(0, 0), scale_limit=(-0.2, 0.2), rotate_limit=(0, 0),p=1.0)],
    bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.2, label_fields=['category_ids']),
)

In [208]:
DEBUG = False

def dataAug(func, imagefiles):
    for imagefile in imagefiles:
        image = cv2.imread(imagefile)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[0:2]
    
        # txt파일은 이미지 파일에서 확장자만 다르다!
        baseName = imagefile.split('.')[0]
        txtFile = baseName+'.txt'
        category_ids, bboxes = read_label_txt(txtFile)
        
        if func=='hflip':
            transformed = transform_hflip(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '_hflip.JPG'  
            saveLabelName = baseName + '_hflip.txt'
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(saveImageName, saveImage)
            write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])            
        elif func=='rotate':
            angle_inter = 20
            for angle in range(angle_inter,360,angle_inter):
                transform_rotate = A.Compose(
                    [A.Rotate(limit=(angle,angle), rotate_method='ellipse', p=1.0)],
                    bbox_params=A.BboxParams(format='yolo', label_fields=['category_ids'])
                )
                transformed = transform_rotate(image=image, bboxes=bboxes, category_ids=category_ids)
                saveImageName = baseName + '_rotate_{}.JPG'.format(angle)  
                saveLabelName = baseName + '_rotate_{}.txt'.format(angle)  
                saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
                cv2.imwrite(saveImageName, saveImage)
                write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])
        elif func=='resize':
            transformed = transform_resize(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '.JPG'  
            saveLabelName = baseName + '.txt'            
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(saveImageName, saveImage)
            write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])
        elif func=='hsv':
            transformed = transform_hsv(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '_hsv.JPG'  
            saveLabelName = baseName + '_hsv.txt'
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(saveImageName, saveImage)
            write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])  
        elif func=='shift':
            transformed = transform_shift(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '_shift.JPG'  
            saveLabelName = baseName + '_shift.txt'
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(saveImageName, saveImage)
            write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])
        elif func=='scale':
            transformed = transform_scale(image=image, bboxes=bboxes, category_ids=category_ids)
            saveImageName = baseName + '_scale.JPG'  
            saveLabelName = baseName + '_scale.txt'
            saveImage = cv2.cvtColor(transformed['image'], cv2.COLOR_RGB2BGR)
            cv2.imwrite(saveImageName, saveImage)
            write_label_txt(saveLabelName, transformed['category_ids'], transformed['bboxes'])             
            
        if DEBUG==True:
            visualize(
                transformed['image'],
                transformed['bboxes'],
                transformed['category_ids'],
                category_id_to_name,
                img_width, img_height
            )
            break

In [209]:
dataAug('resize', imagefiles)
dataAug('hflip', imagefiles)
dataAug('rotate', imagefiles)

In [210]:
imagefiles2 = glob(os.path.join(orgDataPath, '*.JPG'))
dataAug('hsv', imagefiles2)

imagefiles3 = glob(os.path.join(orgDataPath, '*.JPG'))
dataAug('shift', imagefiles3)

imagefiles4 = glob(os.path.join(orgDataPath, '*.JPG'))
dataAug('scale', imagefiles4)

### org폴더의 파일들을 train, valid, test로 나누어서 복사한다. 비율 ( 7 : 1.5 : 1.5 )

In [211]:
allImages = glob(os.path.join(orgDataPath, '*.JPG'))
allLabels = glob(os.path.join(orgDataPath, '*.txt'))
len(allImages), len(allLabels)

(608, 608)

In [212]:
totalCount = len(allImages)

train_ratio = 0.7
valid_ratio = 0.15
test_ratio  = 0.15

train_size = int(totalCount * train_ratio)
valid_size = int(totalCount * valid_ratio)
test_size  = int(totalCount * test_ratio)+1
trainCount, validCount, testCount, len(allImages)

(425, 91, 91, 608)

### 데이터셋 나기기

In [228]:
from torch.utils.data import random_split
train_dataset, valid_dataset, test_dataset = random_split(allImages, [train_size, valid_size, test_size])

In [229]:
train_dataset[0]

'C:/Users/park0/yolov11/customData/org\\sandAce_rotate_140_shift.JPG'

In [230]:
# 폴더 생성
main_dirs = ['train','valid','test'] 
sub_dirs  = ['images', 'labels']

for main_dir in main_dirs:
    for sub_dir in sub_dirs:
        dirname = os.path.join(HOME, main_dir, sub_dir)
        os.makedirs(dirname, exist_ok=True)

        if main_dir=='train' and sub_dir=='images':
            for srcPath in train_dataset:
                basename = os.path.basename(srcPath)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)
        elif main_dir=='train' and sub_dir=='labels':
            for srcPath in train_dataset:
                basename = os.path.basename(srcPath).split('.')[0] + '.txt'
                srcPath = os.path.join(orgDataPath,basename)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)
        elif main_dir=='valid' and sub_dir=='images':
            for srcPath in valid_dataset:
                basename = os.path.basename(srcPath)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)
        elif main_dir=='valid' and sub_dir=='labels':
            for srcPath in valid_dataset:
                basename = os.path.basename(srcPath).split('.')[0] + '.txt'
                srcPath = os.path.join(orgDataPath,basename)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)
        elif main_dir=='test' and sub_dir=='images':
            for srcPath in test_dataset:
                basename = os.path.basename(srcPath)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)
        elif main_dir=='test' and sub_dir=='labels':
            for srcPath in test_dataset:
                basename = os.path.basename(srcPath).split('.')[0] + '.txt'
                srcPath = os.path.join(orgDataPath,basename)
                dstPath = os.path.join(dirname,basename)
                shutil.copy(srcPath, dstPath)  

In [232]:
#!yolo train model=yolo11n.pt data=C:/Users/park0/yolov11/customData/data.yaml epochs=50

In [234]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="C:/Users/park0/yolov11/customData/data.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.9 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.6  Python-3.10.15 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/Users/park0/yolov11/customData/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, em

train: Scanning C:\Users\park0\yolov11\customData\train\labels... 425 images, 317 backgrounds, 0 corr


train: New cache created: C:\Users\park0\yolov11\customData\train\labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\park0\yolov11\customData\valid\labels... 91 images, 67 backgrounds, 0 corrupt:

val: New cache created: C:\Users\park0\yolov11\customData\valid\labels.cache


...

    100/100      2.39G     0.2101      0.649     0.7945          3        640: 100%|██████████| 27/27
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████

                   all         91         24      0.471          1      0.898      0.878



100 epochs completed in 0.174 hours.
Optimizer stripped from runs\detect\train3\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train3\weights\best.pt, 5.5MB

Validating runs\detect\train3\weights\best.pt...
Ultralytics 8.3.6  Python-3.10.15 torch-2.4.1 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)
YOLO11n summary (fused): 238 layers, 2,582,932 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████


                   all         91         24      0.423      0.964      0.919       0.91
             chocoChip          4          4      0.303          1      0.945      0.945
                 gomet          7          7      0.431      0.857      0.758      0.738
          potatoCookie         10         10      0.556          1      0.978       0.96
               sandAce          3          3      0.401          1      0.995      0.995
Speed: 0.5ms preprocess, 2.3ms inference, 0.0ms loss, 1.6ms postprocess per image
Results saved to runs\detect\train3
